In [1]:
# 03_Interactive_Visualizations Notebook
# This notebook allows the user to have an interactive map notebook where they can view play and test constraints of the data on a Florida Bay Web Map visual.

# Import necessary libraries
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

# Load cleaned and original data
data_cleaned = pd.read_csv('../data/cleaned_water_quality.csv')
data_original = pd.read_csv('../data/water_quality.csv')

# Function to preprocess and plot data
def preprocess_and_plot(data, title, file_suffix):
    # Remove outliers using IQR method
    Q1 = data['NO23'].quantile(0.25)
    Q3 = data['NO23'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data = data[(data['NO23'] >= lower_bound) & (data['NO23'] <= upper_bound)]
    
    # Perform min-max normalization on 'NO23' column
    scaler = MinMaxScaler(feature_range=(0, 1))
    data['NO23_normalized'] = scaler.fit_transform(data[['NO23']])
    
    # Create an interactive map with normalized data
    fig = px.scatter_mapbox(data, lat='lat', lon='lon', color='NO23_normalized',
                            hover_data=['station', 'date', 'NO23'],
                            color_continuous_scale='RdBu',
                            range_color=(0, 1),
                            mapbox_style='open-street-map', zoom=10)
    fig.update_layout(title=title,
                      mapbox=dict(center=dict(lat=data['lat'].mean(), lon=data['lon'].mean())))
    fig.write_html(f'../reports/figures/{file_suffix}.html')  # Saving interactive map as HTML
    fig.show()

# Preprocess and plot cleaned data
preprocess_and_plot(data_cleaned, 'Spatial Distribution of Normalized Nitrogen Concentration (Cleaned)', 'cleaned_data_map')

# Preprocess and plot original data
preprocess_and_plot(data_original, 'Spatial Distribution of Normalized Nitrogen Concentration (Original)', 'original_data_map')
